In [1]:
# Generative graphs
%matplotlib inline
import networkx as nx
import pandas as pd

from IPython.display import display, HTML
from collections import defaultdict

from prepare_datasets import preprocess_generative_graph
from graphs import generate_graph, attach_graph_attributes
from measurements import get_graph_measurements, collect_graph_measurements
from graph_to_dataframe import graph_to_training_dataframe, preprocess_dataframe
from recreate_graph import get_trained_model, recreate_by_priority_rank

Using TensorFlow backend.
/home/smoq/.virtualenvs/cna/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
graph_families = [
    {
        'name': 'erdos renyi',
        'func': nx.erdos_renyi_graph,
        'params': [
            {'n': 30, 'p': 0.05},
            {'n': 30, 'p': 0.1},
            {'n': 30, 'p': 0.3}
        ]
    },
    {
        'name': 'watts strogatz',
        'func': nx.watts_strogatz_graph,
        'params': [
            {'n': 30, 'k': 2, 'p': 0.05},
            {'n': 30, 'k': 2, 'p': 0.1},
            {'n': 30, 'k': 2, 'p': 0.3}
        ]
    },
    {
        'name': 'barabasi albert',
        'func': nx.barabasi_albert_graph,
        'params': [
            {'n': 30, 'm': 1},
            {'n': 30, 'm': 2},
            {'n': 30, 'm': 5}
        ]
    },
]

In [3]:
EPOCHS = 16
BATCH_SIZE = 64
VERBOSE = 0

In [4]:
results = defaultdict(list)
for graph_family in graph_families:
    for params in graph_family['params']:
        # Generate graph
        graph = generate_graph(graph_family['func'], params)
        # Set weights to 1 and change graph type to directed
        graph = preprocess_generative_graph(graph)
        # Graph attributes
        attach_graph_attributes(graph)
        # Training df
        df = graph_to_training_dataframe(graph)
        # Preprocess training df
        df = preprocess_dataframe(df, graph.number_of_nodes())
        # Train model
        model = get_trained_model(df, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=VERBOSE)
        
        # drop target column
        X_test = df.drop(['num_of_edges'], axis=1)
        # predict num_edges
        y_pred = model.predict(X_test)
        
        # Recreate graph
        new_graph = recreate_by_priority_rank(graph, y_pred)
        # Get measurements
        graph_measurements = get_graph_measurements(graph)
        new_graph_measurements = get_graph_measurements(new_graph)
        # Make comparison
        #comparison = compare_graph_measurements(graph_measurements, new_graph_measurements)
        measurement_values = collect_graph_measurements(graph_measurements, new_graph_measurements)
        results[graph_family['name']].append(measurement_values)

900/900 [==============================] - 0s 46us/step
loss: 0.20297255840566422, accuracy: 0.9355555555555556
Cannot compute average_shortest_path_length - Graph is not weakly connected.
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected


/home/smoq/.virtualenvs/cna/lib/python3.6/site-packages/networkx/algorithms/assortativity/correlation.py:287: RuntimeWarning: invalid value encountered in double_scalars
  return (xy*(M-ab)).sum()/numpy.sqrt(vara*varb)


900/900 [==============================] - 0s 41us/step
loss: 0.20781820635000864, accuracy: 0.8955555555555555
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected


/home/smoq/.virtualenvs/cna/lib/python3.6/site-packages/networkx/algorithms/assortativity/correlation.py:287: RuntimeWarning: divide by zero encountered in double_scalars
  return (xy*(M-ab)).sum()/numpy.sqrt(vara*varb)


900/900 [==============================] - 0s 37us/step
loss: 0.23371800131267972, accuracy: 0.6844444444444444
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
900/900 [==============================] - 0s 71us/step
loss: 0.20332336319817437, accuracy: 0.9333333333333333
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
900/900 [==============================] - 0s 56us/step
loss: 0.2033068965540992, accuracy: 0.9333333333333333
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
900/900 [==============================] - 0s 105us/step
loss: 0.06487975406977865, accuracy: 0.9333333333333333
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
900/900 [==============================] - 0s 79us/step
loss: 0.06596168827679422, accuracy: 0.9355555555555556
Cannot compute diameter - Found infinite pat

In [5]:
for key, value in results.items():
    print(key)
    for i, comparison_result in enumerate(value):
        print('graph #{}'.format(i+1))
        results_df = pd.DataFrame([comparison_result])
        #print(results_df)
        display(HTML(results_df.to_html()))

erdos renyi
graph #1


,degree_centrality_ks_stat,degree_centrality_ks_pval,degree_centrality_ent_pred,degree_centrality_kl_div,closeness_centrality_ks_stat,closeness_centrality_ks_pval,closeness_centrality_ent_pred,closeness_centrality_kl_div,betweenness_centrality_ks_stat,betweenness_centrality_ks_pval,betweenness_centrality_ent_pred,betweenness_centrality_kl_div,pagerank_ks_stat,pagerank_ks_pval,pagerank_ent_pred,pagerank_kl_div,average_shortest_path_length,diameter,degree_centralization,closeness_centralization,betweenness_centralization,pagerank_centralization,density,degree_assortativity,reciprocity,transitivity
0,0.2,0.5372,3.2283,0.426531,0.366667,0.025856,2.770396,inf,0.266667,0.200325,2.645385,inf,0.533333,0.000202,2.779655,0.863513,1.263218,None,24.545455,21.306631,25.465696,24.474926,0.068966,NaN,0.033333,0.160714


graph #2


,degree_centrality_ks_stat,degree_centrality_ks_pval,degree_centrality_ent_pred,degree_centrality_kl_div,closeness_centrality_ks_stat,closeness_centrality_ks_pval,closeness_centrality_ent_pred,closeness_centrality_kl_div,betweenness_centrality_ks_stat,betweenness_centrality_ks_pval,betweenness_centrality_ent_pred,betweenness_centrality_kl_div,pagerank_ks_stat,pagerank_ks_pval,pagerank_ent_pred,pagerank_kl_div,average_shortest_path_length,diameter,degree_centralization,closeness_centralization,betweenness_centralization,pagerank_centralization,density,degree_assortativity,reciprocity,transitivity
0,0.3,0.108838,3.255103,0.237167,0.4,0.011314,3.11174,inf,0.166667,0.760047,2.916797,inf,0.466667,0.001746,2.946257,0.568296,2.406897,None,23.571429,15.984956,20.643377,23.344407,0.103448,inf,0.088889,0.210227


graph #3


...

,degree_centrality_ks_stat,degree_centrality_ks_pval,degree_centrality_ent_pred,degree_centrality_kl_div,closeness_centrality_ks_stat,closeness_centrality_ks_pval,closeness_centrality_ent_pred,closeness_centrality_kl_div,betweenness_centrality_ks_stat,betweenness_centrality_ks_pval,betweenness_centrality_ent_pred,betweenness_centrality_kl_div,pagerank_ks_stat,pagerank_ks_pval,pagerank_ent_pred,pagerank_kl_div,average_shortest_path_length,diameter,degree_centralization,closeness_centralization,betweenness_centralization,pagerank_centralization,density,degree_assortativity,reciprocity,transitivity
0,0.333333,0.054993,3.2283,0.175779,0.666667,0.000001,2.741217,inf,0.3,0.108838,2.628677,inf,0.666667,0.000001,2.705719,0.595563,1.051724,None,24.545455,21.47052,24.905325,25.072114,0.068966,NaN,0.066667,0.224138


graph #2


,degree_centrality_ks_stat,degree_centrality_ks_pval,degree_centrality_ent_pred,degree_centrality_kl_div,closeness_centrality_ks_stat,closeness_centrality_ks_pval,closeness_centrality_ent_pred,closeness_centrality_kl_div,betweenness_centrality_ks_stat,betweenness_centrality_ks_pval,betweenness_centrality_ent_pred,betweenness_centrality_kl_div,pagerank_ks_stat,pagerank_ks_pval,pagerank_ent_pred,pagerank_kl_div,average_shortest_path_length,diameter,degree_centralization,closeness_centralization,betweenness_centralization,pagerank_centralization,density,degree_assortativity,reciprocity,transitivity
0,0.433333,0.004607,3.262115,0.139183,0.466667,0.001746,3.215723,inf,0.366667,0.025856,2.985405,inf,0.533333,0.000202,2.967125,0.215679,2.202299,None,24.285714,15.535622,21.422958,23.075845,0.137931,NaN,0.166667,0.265432


graph #3


,degree_centrality_ks_stat,degree_centrality_ks_pval,degree_centrality_ent_pred,degree_centrality_kl_div,closeness_centrality_ks_stat,closeness_centrality_ks_pval,closeness_centrality_ent_pred,closeness_centrality_kl_div,betweenness_centrality_ks_stat,betweenness_centrality_ks_pval,betweenness_centrality_ent_pred,betweenness_centrality_kl_div,pagerank_ks_stat,pagerank_ks_pval,pagerank_ent_pred,pagerank_kl_div,average_shortest_path_length,diameter,degree_centralization,closeness_centralization,betweenness_centralization,pagerank_centralization,density,degree_assortativity,reciprocity,transitivity
0,0.2,0.5372,3.326902,0.151414,0.366667,0.025856,3.373421,0.024633,0.5,0.000616,3.245558,0.69219,0.466667,0.001746,3.142165,0.243662,1.94023,4,23.076923,21.105354,20.888911,21.511395,0.275862,NaN,0.25,0.450255
